In [6]:
# import os
# dirname_number = 0
# file_number = 1
# writefile = open('NewGroup_data.txt', 'w')
# for dirname, subdir, filename in os.walk('C:/Users/karis/Desktop/latently/20news-bydate-train/', topdown=True):
#     if dirname != "C:/Users/karis/Desktop/latently/20news-bydate-train/":
#         for file_nam in filename:
#             f = os.path.join(dirname, file_nam)
#             with open(f,'r') as main:
#                 contents = main.readlines()
#                 writefile.write("{}\t{}\t{}\n".format(file_number, contents, dirname_number))
#                 file_number = file_number+1
#         dirname_number = dirname_number+1
# writefile.close()


In [9]:
group_names = []
for dirname, subdir, filename in os.walk('C:/Users/karis/Desktop/latently/20news-bydate-train/', topdown=False):
     for i in  subdir:
            group_names.append(i)
group_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [149]:
import pandas as pd
import numpy as np
import os
from pandas import Series, DataFrame
group_train = pd.read_csv("C:\Users\karis\NewGroup_data.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_train.head()

,phrase_id,review,group_id
0,1,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
1,2,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
2,3,['From: I3150101@dbstu1.rz.tu-bs.de (Benedikt ...,0
3,4,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
4,5,"['From: strom@Watson.Ibm.Com (Rob Strom)\n', '...",0


In [150]:
group_test = pd.read_csv("C:\Users\karis\NewGroup_data_test.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_test.head()

,phrase_id,review,group_id
0,1,['From: decay@cbnewsj.cb.att.com (dean.kaflowi...,0
1,2,"['From: cfaehl@vesta.unm.edu (Chris Faehl)\n',...",0
2,3,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
3,4,"['From: dps@nasa.kodak.com (Dan Schaertel,,,)\...",0
4,5,"['From: halat@panther.bears (Jim Halat)\n', 'S...",0


In [151]:
# maping cateogories to train dataframe

group_train['category'] = group_train.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

# maping cateogories to test dataframe

group_test['category'] = group_test.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

group_test.shape

(7532, 4)

In [152]:
# selecting categories 
group_train_category = group_train[(group_train.category == 'comp') | (group_train.category == 'rec') | (group_train.category == 'politics') | (group_train.category == 'religion')]
group_test_category = group_test[(group_test.category == 'comp') | (group_test.category == 'rec') | (group_test.category == 'politics') | (group_test.category == 'religion')]


# spliting data into input X_train and output Y_train
X_train = group_train_category.review
# Y_train = group_train_category.category

# spliting data into input X_train and output Y_train
X_test = group_test_category.review
# Y_test = group_test_category.category

print(X_train.shape)
print(X_test.shape)

(7876L,)
(5244L,)


In [153]:
# use CountVectorizer to create document-term matrices from X_train and X_test

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()


# fit and transform X_train

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)


print(X_train_dtm.shape)
print(X_test_dtm.shape)



(7876, 111641)
(5244, 111641)


In [157]:
from sklearn.preprocessing import LabelEncoder


Label_encoded = LabelEncoder()
Y_train_label_encoded = Label_encoded.fit_transform(group_train_category['category'].values)
Y_test_label_encoded = Label_encoded.fit_transform(group_test_category['category'].values)


# Y_train_label_encoded = np.vstack(Y_train_label_encoded)
# Y_test_label_encoded = np.vstack(Y_test_label_encoded)

print(Y_train_label_encoded.shape)
print(Y_test_label_encoded.shape)



(7876L,)
(5244L,)


In [163]:
def tokenize_test(vect):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    LR = LogisticRegression(multi_class= 'ovr')

    LR.fit(X_train_dtm, Y_train_label_encoded)

    Y_predicted = LR.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test_label_encoded, Y_predicted))



In [164]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test(vect)

('accracy:', 0.94508009153318073)


In [165]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

('accracy:', 0.9416475972540046)


In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tokenize_test(vect)

('accracy:', 0.93821510297482835)


In [182]:
def tokenize_test_svm(vect):
    from sklearn import svm
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train_dtm, Y_train_label_encoded)
    Y_predicted = lin_clf.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test_label_encoded, Y_predicted))

In [183]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test_svm(vect)

('accracy:', 0.93688024408848203)


In [184]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test_svm(vect)

('accracy:', 0.94107551487414187)


In [179]:
# from sklearn import svm

# lin_clf = svm.LinearSVC()
# lin_clf.fit(X_train_dtm, Y_train_label_encoded)
# Y_predicted = lin_clf.predict(X_test_dtm)

In [180]:
# from sklearn.metrics import accuracy_score

# print ('accracy:', accuracy_score(Y_test_label_encoded, Y_predicted))

('accracy:', 0.93688024408848203)
